# 02_07: Connecting Python to Neo4j

In [2]:
from neo4j import GraphDatabase

In [3]:
URI = "neo4j+s://6888654b.databases.neo4j.io"
USER = "neo4j"
PWD = "uCLAEnOsMD46L4Qn8T2q2JzZii1YBJ0fikwMzeQG0Ts"
AUTH = (USER, PWD)

## A note about this next code block

There are many ways to connect to Neo4j from Python.  This here is a little helper class that manages the connection for you and includes some basic error handling.  We will use this class in pretty much every notebook that needs to connect to Neo4j in this course, but this is what it looks like.

In [4]:
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [5]:
neo_conn = Neo4jConnection(uri=URI, user=USER, pwd=PWD)

In [6]:
neo_conn.query('MATCH (n) RETURN COUNT(n) AS total')

[<Record total=1104>]

In [7]:
neo_conn.query('MATCH(a:Employee)-[r:REPORTS_TO]-(b) RETURN a, r, b')

[<Record a=<Node element_id='4:365c6d96-c430-481b-844d-f152aaaf564a:1095' labels=frozenset({'Employee'}) properties={'country': 'USA', 'lastName': 'Davolio', 'hireDate': '1992-05-01 00:00:00.000', 'extension': '5467', 'address': '507 - 20th Ave. E. Apt. 2A', 'notes': 'Education includes a BA in psychology from Colorado State University in 1970.  She also completed "The Art of the Cold Call."  Nancy is a member of Toastmasters International.', 'city': 'Seattle', 'photoPath': 'http://accweb/emmployees/davolio.bmp', 'postalCode': '98122', 'homePhone': '(206) 555-9857', 'employeeID': '1', 'title': 'Sales Representative', 'titleOfCourtesy': 'Ms.', 'birthDate': '1948-12-08 00:00:00.000', 'firstName': 'Nancy', 'region': 'WA'}> r=<Relationship element_id='5:365c6d96-c430-481b-844d-f152aaaf564a:1152928101676614727' nodes=(<Node element_id='4:365c6d96-c430-481b-844d-f152aaaf564a:1095' labels=frozenset({'Employee'}) properties={'country': 'USA', 'lastName': 'Davolio', 'hireDate': '1992-05-01 00:0